# Tâche 5 : Analyse des symétries 

In [1]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
mp_key = "MgnCbwKILRoKeWCsa2tbfn04aBTHkB78"
mp_id = "mp-1747"

In [3]:
with MPRester(mp_key) as m:
    print("Téléchargement du fichier CIF de K2Te..")
    prim_struc = m.get_structure_by_material_id(mp_id)
    print("Téléchargement terminé")
symmops = SpacegroupAnalyzer(prim_struc).get_symmetry_operations()
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()

Téléchargement du fichier CIF de K2Te..


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Téléchargement terminé


 ## Choix des atomes et des symétries

Dans ce notebook, nous allons nous intéresser à l'analyse de quelques symétrie présente dans la structure cristalline du $K_2Te$. Les opérations analysées sont : une roto-inversion d'ordre 4 ($\overline{4}$ en notation de Hermann-Maugin ou $iC_4$ en notation de Schönflies), une rotation autour d'un axe d'ordre 3 ($3$ ou $C_3$) et une inversion ($\overline{1}$ ou $i$). Ces 3 opérations sont des symétries dites ponctuelles c'est à dire que au moins un point de la maille reste fixe selon la symétrie.

In [4]:
#Choix des atomes:
pos_init_Te = conv_struc.sites[10].frac_coords
pos_init_K1 = conv_struc.sites[7].frac_coords
pos_init_K2 = conv_struc.sites[1].frac_coords

#Plan miroir xy (x, z, y)
s1 = symmops[31]
Te_symmetry = s1.operate(pos_init_Te)

#Roto-inversion (-x,-z,-y)
s2 = symmops [28]
K1_symmetry = s2.operate(pos_init_K1)

#Rotation axe d'ordre 2 (y, x, -x-y-z)
s3 = symmops [14]
K2_symmetry = s3.operate(pos_init_K2)

print("Les atomes étudiés sont : Te situé en {0}, K1 situé en {1} et K2 situé en {2}. \n\nLes symétries étudiées sont : une roto-inversion d'ordre 4 sur l'atome de Te, une rotation autour d'un axe \nd'ordre 3 sur K1 et une inversion pour K2.".format(pos_init_Te, pos_init_K1, pos_init_K2))

Les atomes étudiés sont : Te situé en [0.5 0.  0.5], K1 situé en [0.25 0.25 0.25] et K2 situé en [0.75 0.75 0.25]. 

Les symétries étudiées sont : une roto-inversion d'ordre 4 sur l'atome de Te, une rotation autour d'un axe 
d'ordre 3 sur K1 et une inversion pour K2.


## Symétrie 1 : Roto-Inversion d'ordre 4 

In [5]:
print(" Nous pouvons voir dans la figure ci dessous que l'atome de Tellure initialement \n situé aux coordonnées : {0} subit une symétrie (x, z, y) constituée : \n - D'une inversion autour du point [0.0, 0.0, 0.0] \n - D'une rotation d'ordre 4 autour de l'axe (0 1 0)  \n et arrive au final à la position {1}. \n\n L'opération de symétrie est donnée par la matrice de rotation : \n {2} \n et le vecteur de translation {3} ".format(pos_init_Te, Te_symmetry, s1.rotation_matrix, s1.translation_vector))
print(" \nCette opération est une opération de seconde espèce car le déterminant de la matrice \nvaut -1. Cette opération ne conserve donc pas la chiralité du cristal.")
v1 = quick_view(conv_struc, conventional = "True")
display(v1)
v1.script("draw SYMOP 32 {atomno=19}")



 Nous pouvons voir dans la figure ci dessous que l'atome de Tellure initialement 
 situé aux coordonnées : [0.5 0.  0.5] subit une symétrie (x, z, y) constituée : 
 - D'une inversion autour du point [0.0, 0.0, 0.0] 
 - D'une rotation d'ordre 4 autour de l'axe (0 1 0)  
 et arrive au final à la position [0.5 0.5 0. ]. 

 L'opération de symétrie est donnée par la matrice de rotation : 
 [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]] 
 et le vecteur de translation [0. 0. 0.] 
 
Cette opération est une opération de seconde espèce car le déterminant de la matrice 
vaut -1. Cette opération ne conserve donc pas la chiralité du cristal.


JsmolView(layout=Layout(align_self='stretch', height='400px'))

## Symétrie 2 : Rotation autour d'un axe d'ordre 3 (-x, -z, -y)

In [6]:
print(" Nous pouvons voir dans la figure ci dessous que l'atome de Potassium n°1 initialement \n situé aux coordonnées : {0} subit une inversion selon un axe d'ordre 3 pour \n arriver à la position {1}. \n\n L'opération de symétrie est donnée par la matrice de rotation : \n {2} \n et le vecteur de translation {3} ".format(pos_init_K1, K2_symmetry, s2.rotation_matrix, s2.translation_vector))
print(" \n Cette opération est une opération de première espèce car le déterminant de la matrice \n vaut 1. Cette opération conserve donc la chiralité du cristal.")
v2 = quick_view(conv_struc, conventional = "True")
display(v2)
v2.script("draw SYMOP 29 {atomno=1}")


 Nous pouvons voir dans la figure ci dessous que l'atome de Potassium n°1 initialement 
 situé aux coordonnées : [0.25 0.25 0.25] subit une inversion selon un axe d'ordre 3 pour 
 arriver à la position [ 0.25  0.75 -1.75]. 

 L'opération de symétrie est donnée par la matrice de rotation : 
 [[-1.  0.  0.]
 [ 0.  0. -1.]
 [ 0. -1.  0.]] 
 et le vecteur de translation [0. 0. 0.] 
 
 Cette opération est une opération de première espèce car le déterminant de la matrice 
 vaut 1. Cette opération conserve donc la chiralité du cristal.


JsmolView(layout=Layout(align_self='stretch', height='400px'))

## Symétrie 3 : Inversion (z, x, -x-y-z)

In [7]:
print(" Nous pouvons voir dans la figure ci dessous que l'atome de Potassium n°2 initialement \n situé aux coordonnées : {0} subit une une inversion autour du point [0.75, 0.75, 0.0] pour \n arriver à la position {1}. \n\n L'opération de symétrie est donnée par la matrice de rotation : \n {2} \n et le vecteur de translation {3} ".format(pos_init_K2, K2_symmetry, s3.rotation_matrix, s3.translation_vector))
print(" \n Cette opération est une opération de seconde espèce car le déterminant de la matrice \n vaut -1. Cette opération ne conserve donc pas la chiralité du cristal.")
v3 = quick_view(conv_struc, conventional = "True")
display(v3)
v3.script("draw SYMOP 15 {atomno=9}")


 Nous pouvons voir dans la figure ci dessous que l'atome de Potassium n°2 initialement 
 situé aux coordonnées : [0.75 0.75 0.25] subit une une inversion autour du point [0.75, 0.75, 0.0] pour 
 arriver à la position [ 0.25  0.75 -1.75]. 

 L'opération de symétrie est donnée par la matrice de rotation : 
 [[ 0.  0.  1.]
 [ 1.  0.  0.]
 [-1. -1. -1.]] 
 et le vecteur de translation [0. 0. 0.] 
 
 Cette opération est une opération de seconde espèce car le déterminant de la matrice 
 vaut -1. Cette opération ne conserve donc pas la chiralité du cristal.


JsmolView(layout=Layout(align_self='stretch', height='400px'))